Updated notebook - main changes: revert to separating each category into its own collection, implement 2 step pipeline (categorize, then retrieve from those relevant collections). Somehow, takes a substantial amount of time now (ie 7 queries in 17 minutes) (using general qa set and some other random set of data), to discuss.

## Please Download the gguf file (model_path)

download this: Llama-3.2-1B-Instruct-IQ3_M.gguf
link: https://huggingface.co/bartowski/Llama-3.2-1B-Instruct-GGUF/blob/main/Llama-3.2-1B-Instruct-IQ3_M.gguf

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
#!pip install --upgrade --upgrade-strategy eager "optimum[neural-compressor]"
!pip install llama-cpp-python

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

import time
import uuid
from llama_cpp import Llama
#from fuzzywuzzy import fuzz
#from optimum.quantization import QuantizeruenceClassification


Define Variables

In [ ]:
from llama_cpp import Llama

model_path = "Llama-3.2-1B-Instruct-IQ3_M.gguf"

model = Llama(model_path=model_path, n_ctx=2048, n_threads=8)


In [ ]:
#ls -lh /content/Llama-3.2-1B-Instruct-IQ3_M.gguf

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import uuid
import pandas as pd
#from fuzzywuzzy import fuzz

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")



Define Data Sources

In [6]:
import pandas as pd
import concurrent.futures
import uuid
import os

file_names = [
    "study_permit_general", "work_permit_student_general", "work-study-data-llm",
    "vancouver_transit_qa_pairs", "permanent_residence_student_general", "data-with-sources",
    "faq_qa_pairs_general", "hikes_qa", "sfu-faq-with-sources", "sfu-housing-with-sources",
    "sfu-immigration-faq", "park_qa_pairs-up", "cultural_space_qa_pairs_up",
    "qa_pairs_food", "qa_pairs_year_and_month_avg", "qa_pairs_sfu_clubs"
]

collections = {}
batch_size = 32

def process_file(file):
    try:
        path = f'../Data/{file}.csv'
        if not os.path.exists(path):
            return f"{file} skipped (file not found)."

        df = pd.read_csv(path, usecols=lambda col: col.lower() in {"question", "answer"})
        df.columns = df.columns.str.lower()

        if "question" not in df.columns or "answer" not in df.columns:
            return f"{file} skipped (missing question/answer columns)."

        df = df.drop_duplicates(subset="question")
        df["text"] = df["question"].fillna('') + ' ' + df["answer"].fillna('')
        unique_texts = list(set(df["text"].dropna().tolist()))

        collection = client.get_or_create_collection(name=file)
        for i in range(0, len(unique_texts), batch_size):
            batch = unique_texts[i:i + batch_size]
            embeddings = embedding_model.embed_documents(batch)
            ids = [str(uuid.uuid4()) for _ in batch]
            collection.add(ids=ids, embeddings=embeddings, documents=batch)

        collections[file] = collection
        return f"{file}: Loaded {len(unique_texts)} docs."
    except Exception as e:
        return f"{file}: Error - {e}"

# parallelogram
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    results = list(executor.map(process_file, file_names))

for result in results:
    print(result)


study_permit_general: Loaded 14 docs.
work_permit_student_general: Loaded 23 docs.
work-study-data-llm: Loaded 178 docs.
vancouver_transit_qa_pairs: Loaded 37 docs.
permanent_residence_student_general: Loaded 6 docs.
data-with-sources: Loaded 75 docs.
faq_qa_pairs_general: Loaded 220 docs.
hikes_qa: Loaded 278 docs.
sfu-faq-with-sources: Loaded 102 docs.
sfu-housing-with-sources: Loaded 74 docs.
sfu-immigration-faq: Loaded 83 docs.
park_qa_pairs-up: Loaded 216 docs.
cultural_space_qa_pairs_up: Loaded 560 docs.
qa_pairs_food: Loaded 38 docs.
qa_pairs_year_and_month_avg: Loaded 1488 docs.
qa_pairs_sfu_clubs: Loaded 163 docs.


In [ ]:
# define cat to collection mapping
# motivation: takes wayyyy too long now -> dataset size trippled and time grew exponentially...
# now takes around 2 mins on avg for each response..compared to 40 seconds previosusly..

collection_map = {
    "study": "study_permit_general",
    "student work": "work_permit_student_general",
    "work-study": "work-study-data-llm",
    "transit": "vancouver_transit_qa_pairs",
    "permanent residence": "permanent_residence_student_general",
    "general info": "data-with-sources",
    "faq": "faq_qa_pairs_general",
    "hiking": "hikes_qa",
    "sfu faq": "sfu-faq-with-sources",
    "housing": "sfu-housing-with-sources",
    "immigration": "sfu-immigration-faq",
    "parks": "park_qa_pairs-up",
    "culture": "cultural_space_qa_pairs_up",
    "food": "qa_pairs_food",
    "weather": "qa_pairs_year_and_month_avg",
    "clubs": "qa_pairs_sfu_clubs"
}


Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [ ]:
'''
# seems like better results if we remove duplicates and very similar data
data = pd.DataFrame({"text": all_texts})
data = data.drop_duplicates()
all_texts = data["text"].tolist()

print(f"successfully added {len(all_texts)} documents to Chroma DB.")
'''

Function to add data to collection by embedding them

In [27]:
'''
def add_data_to_collection_batch(collection, texts, batch_size=3):
    for idx in range(0, len(texts), batch_size):
        try:
            batch_texts = texts[idx: idx + batch_size]

            embeddings = embedding_model.embed_documents(batch_texts)

            batch_ids = [str(uuid.uuid4()) for _ in batch_texts]

            collection.add(
                ids=batch_ids,
                embeddings=embeddings,
                documents=batch_texts
            )
            print(f"successfully added {len(batch_texts)} documents (Batch {idx}-{idx + batch_size - 1})")
        except Exception as e:
            print(f"Error processing batch starting at index {idx}: {e}")
'''

Function to now match for releveant document

In [7]:
def get_relevant_documents(query, categories, n_results=2):
    all_results = []
    query_embedding = embedding_model.embed_documents([query])[0]

    for category in categories:
        collection_name = collection_map[category]
        if collection_name in collections:
            try:
                result = collections[collection_name].query(
                    query_embeddings=[query_embedding],
                    n_results=n_results
                )
                docs = result.get("documents", [[]])[0]
                sims = result.get("distances", [[]])[0]

                all_results.extend(zip(docs, sims))
            except Exception as e:
                print(f"error querying {collection_name}: {e}")

    all_results = sorted(all_results, key=lambda x: x[1])

    return all_results[:n_results]


### Classify Prompt

In [ ]:
import re
import difflib

valid_categories = list(collection_map.keys())
fallback_category = "faq"

def classify_query(query):
    category_prompt = f"""
    You are a classifier for a Q&A system for international students in British Columbia.
    Choose the **1 most relevant** category from this list, or at most 3 if absolutely needed (comma-separated):

    {", ".join(valid_categories)}

    Query: "{query}"

    Return only the category name(s) as a comma-separated string.
    """

    response = model(category_prompt, max_tokens=50, temperature=0)["choices"][0]["text"].strip().lower()
    print("Raw out:", response)

    tokens = re.findall(r'\b\w+\b', response)

    matched = []
    for token in tokens:
        closest = difflib.get_close_matches(token, valid_categories, n=1, cutoff=0.8)
        if closest and closest[0] not in matched:
            matched.append(closest[0])
        if len(matched) == 3:
            break

    if fallback_category not in matched:
      matched.append(fallback_category)

    return matched[:3]


Generate Answer

In [64]:
def generate_answer(query):
    categories = classify_query(query)
    print(f"Categories {categories}\n")
    relevant_documents = get_relevant_documents(query, categories)

    if not relevant_documents:
        return {
            "After RAG Response": "Sorry, no relevant documents found."
        }

    #relevant_documents = list(set(relevant_documents))

    seen = set()
    unique_docs = []
    for doc, sim in relevant_documents:
        if doc not in seen:
            seen.add(doc)
            unique_docs.append((doc, sim))

    print("Relevant Documents with Similarity Scores:")
    for doc, sim in unique_docs:
        print(f"Similarity: {sim:.4f}\nDoc: {doc}\n")

    relevant_texts = "\n\n".join([doc for doc, _ in unique_docs])

    rag_prompt = f"""
    You are a helpful assistant for international students new to British Columbia Canada. Here are relevant documents:

    {relevant_texts}

    Please respond to the following question based on the documents above. Be conversational but concise, aim to answer accurately using the documents, but in as few words as possible (i.e. less than 20).
    Question: {query}

    Answer:
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]

    return {
        "After RAG Response": response_after_rag
    }


Example Usage

In [69]:
import pandas as pd

benchmark_data = pd.read_csv("./sample_data/seen-data.csv")

for idx, row in benchmark_data.iterrows():
    user_query = row["Question"]
    correct_answer = row["Answer"]

    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Benchmark Query {idx + 1}: {user_query}")
    print("="*50)
    print("\nResponse After RAG:\n", responses.get("After RAG Response", "N/A"))
    print("\n(Benchmark) Answer:\n", correct_answer)
    print("="*50 + "\n\n")


Llama.generate: 2 prefix-match hit, remaining 108 prompt tokens to eval
llama_perf_context_print:        load time =   95543.24 ms
llama_perf_context_print: prompt eval time =  170802.70 ms /   576 tokens (  296.53 ms per token,     3.37 tokens per second)
llama_perf_context_print:        eval time =   14250.28 ms /    49 runs   (  290.82 ms per token,     3.44 tokens per second)
llama_perf_context_print:       total time =   39082.24 ms /   625 tokens
Llama.generate: 2 prefix-match hit, remaining 267 prompt tokens to eval


Raw out: the query is likely to be related to arts and cultural events, so i would recommend the following categories:

arts, cultural, events, festivals, concerts, performances, exhibitions, museums, arts, cultural events, arts festivals, cultural events, cultural events
Categories ['culture', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 21.2244
Doc: Where can I find information about arts and cultural events? You can visit theDestination British Columbiawebsite, which has information about art galleries, museums and heritage sites across British Columbia. The British Columbia Arts Council also publishes a searchable calendar of arts and cultural events in communities across British Columbia. Some places and events will cost money, but they may be free or give a discount on certain days. Some communities have their own special events, such as festivals and fairs. These are often free and give you a chance to learn about your community. You can get information at publ

llama_perf_context_print:        load time =   95543.24 ms
llama_perf_context_print: prompt eval time =   59614.45 ms /   267 tokens (  223.28 ms per token,     4.48 tokens per second)
llama_perf_context_print:        eval time =   19381.33 ms /    64 runs   (  302.83 ms per token,     3.30 tokens per second)
llama_perf_context_print:       total time =   79095.52 ms /   331 tokens
Llama.generate: 2 prefix-match hit, remaining 111 prompt tokens to eval



Benchmark Query 1: Where can I find information about arts and cultural events?

Response After RAG:
  You can find information about arts and cultural events in British Columbia through the following sources:
     Destination British Columbia website
     British Columbia Arts Council
     Public libraries
     Tourist information offices
     Arts councils
     Municipal park boards
     Destination British Columbia Things to Do – Arts & Heritage

    Good luck!

(Benchmark) Answer:
 Visit the Destination British Columbia website for art galleries, museums, and heritage sites. The British Columbia Arts Council also offers a searchable calendar. Some events are free or discounted on certain days, and local libraries, tourist offices, and municipal park boards often provide event details.




llama_perf_context_print:        load time =   95543.24 ms
llama_perf_context_print: prompt eval time =   25412.97 ms /   111 tokens (  228.95 ms per token,     4.37 tokens per second)
llama_perf_context_print:        eval time =   14233.55 ms /    49 runs   (  290.48 ms per token,     3.44 tokens per second)
llama_perf_context_print:       total time =   39722.63 ms /   160 tokens
Llama.generate: 2 prefix-match hit, remaining 205 prompt tokens to eval


...

llama_perf_context_print:        load time =   95543.24 ms
llama_perf_context_print: prompt eval time =   76294.92 ms /   333 tokens (  229.11 ms per token,     4.36 tokens per second)
llama_perf_context_print:        eval time =   30310.13 ms /   103 runs   (  294.27 ms per token,     3.40 tokens per second)
llama_perf_context_print:       total time =  106769.94 ms /   436 tokens
Llama.generate: 2 prefix-match hit, remaining 108 prompt tokens to eval



Benchmark Query 7: What do I need to know about working in Canada?

Response After RAG:
  You need to apply for a Social Insurance Number (SIN) in order to work in Canada or to have access to government programs and benefits. Finding a job in Canada may be different from finding a job in your home country. You may face challenges at the beginning to get a job that matches your qualifications and interests. It may take time to build your qualifications and gain Canadian experience before finding the job you really want. However, there are several resources that can help you understand what to do to find a job.

(Benchmark) Answer:
 Apply for a Social Insurance Number (SIN) to work and access government programs. The job market might require building Canadian experience, but resources are available to assist with job searches.




llama_perf_context_print:        load time =   95543.24 ms
llama_perf_context_print: prompt eval time =   24776.07 ms /   108 tokens (  229.41 ms per token,     4.36 tokens per second)
llama_perf_context_print:        eval time =   14283.12 ms /    49 runs   (  291.49 ms per token,     3.43 tokens per second)
llama_perf_context_print:       total time =   39136.89 ms /   157 tokens
Llama.generate: 2 prefix-match hit, remaining 227 prompt tokens to eval


Raw out: answer: sin, student work, faq, transit, general info, faq, housing, immigration, parks, culture, food, weather, clubs, faq, faq, faq, faq, faq, faq,
Categories ['faq', 'transit', 'housing']

Relevant Documents with Similarity Scores:
Similarity: 14.8552
Doc: What is a Social Insurance Number (SIN)? A Social Insurance Number (SIN) is a nine-digit number that you need in order to work in Canada or have access to government programs and benefits. Children aged 12 years and older may apply for their own SIN. Parents and legal guardians can also apply for a SIN for children under the age of majority in their province. Each SIN is issued to one person only. It cannot legally be used by anyone else. You are responsible for protecting your SIN, so it is important that you keep it in a safe place. For more information: WelcomeBC: Social Insurance Number WelcomeBC: Social Insurance Number Government of Canada: Social Insurance Number Government of Canada: Social Insurance Number 



KeyboardInterrupt: 

In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())


2
